In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_name = "Qwen/Qwen2.5-1.5B" 
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [3]:
sd = model.state_dict()
for k, v in sd.items():
    print(k, v.shape)

model.embed_tokens.weight torch.Size([151936, 1536])
model.layers.0.self_attn.q_proj.weight torch.Size([1536, 1536])
model.layers.0.self_attn.q_proj.bias torch.Size([1536])
model.layers.0.self_attn.k_proj.weight torch.Size([256, 1536])
model.layers.0.self_attn.k_proj.bias torch.Size([256])
model.layers.0.self_attn.v_proj.weight torch.Size([256, 1536])
model.layers.0.self_attn.v_proj.bias torch.Size([256])
model.layers.0.self_attn.o_proj.weight torch.Size([1536, 1536])
model.layers.0.mlp.gate_proj.weight torch.Size([8960, 1536])
model.layers.0.mlp.up_proj.weight torch.Size([8960, 1536])
model.layers.0.mlp.down_proj.weight torch.Size([1536, 8960])
model.layers.0.input_layernorm.weight torch.Size([1536])
model.layers.0.post_attention_layernorm.weight torch.Size([1536])
model.layers.1.self_attn.q_proj.weight torch.Size([1536, 1536])
model.layers.1.self_attn.q_proj.bias torch.Size([1536])
model.layers.1.self_attn.k_proj.weight torch.Size([256, 1536])
model.layers.1.self_attn.k_proj.bias tor